In [ ]:
import os
import logging
from pathlib import Path
import mimetypes
from dotenv import load_dotenv
load_dotenv()

import boto3
from botocore.exceptions import ClientError
from botocore.client import Config

BUCKET_NAME = 'urban-heat-files'
DATA_FOLDER = Path('../_data/final/')

In [ ]:
# setup s3 
def s3_client(aws_id, aws_key):
	s3 = boto3.client(
		's3',
		aws_access_key_id=aws_id,
		aws_secret_access_key=aws_key,
		config=Config(connect_timeout=10, read_timeout=100, retries={'max_attempts': 10})
	)
	return s3

def put_object(s3_client, dest_bucket_name, dest_object_name, src_data, content_type, cache_control=''):
	"""Add an object to an Amazon S3 bucket

    https://docs.aws.amazon.com/code-samples/latest/catalog/python-s3-put_object.py.html

	The src_data argument must be of type bytes or a string that references
	a file specification.

    :param s3_client: s3_client as returned by s3_client()
	:param dest_bucket_name: string
	:param dest_object_name: string
	:param src_data: bytes of data or string reference to file spec
	:param content_type: string to set for Content-Type header on file
	:param cache_control: string to set for Cache-Control header on file (if not provided, header not set)
	:return: True if src_data was added to dest_bucket/dest_object, otherwise
	False
	"""
	# Construct Body= parameter
	if isinstance(src_data, bytes):
		object_data = src_data
	elif isinstance(src_data, str):
		try:
			object_data = open(src_data, 'rb')
			# possible FileNotFoundError/IOError exception
		except Exception as e:
			logging.error(e)
			return False
	else:
		logging.error('Type of ' + str(type(src_data)) +
					  ' for the argument \'src_data\' is not supported.')
		return False

	kwargs = {
		'Bucket': dest_bucket_name, 
		'Key': dest_object_name, 
		'Body': object_data,
		'ContentType': content_type
	}

	if cache_control != '':
		kwargs['CacheControl'] = cache_control

	try:
		s3_client.put_object(**kwargs)
	except ClientError as e:
		# AllAccessDisabled error == bucket not found
		# NoSuchKey or InvalidRequest error == (dest bucket/obj == src bucket/obj)
		logging.error(e)
		return False
	finally:
		if isinstance(src_data, str):
			object_data.close()
	return True
    
class S3:
	"""AWS S3 client for getting/putting objects to/from oca-data bucket"""
	def __init__(self, aws_id, aws_key, aws_bucket_name):
		self.s3 = s3_client(aws_id, aws_key)
		self.bucket_name = aws_bucket_name

	def upload_file(self, object_name, file_path, content_type):
		ext = os.path.splitext(file_path)[1]

		# date-updated image needs to have no-cache to be used in github readme
		cache_control = 'no-cache' if content_type == 'image/png' else ''

		# Put the object into the bucket
		put_object(self.s3, self.bucket_name, object_name, str(file_path), content_type, cache_control)

In [ ]:
s3 = S3(os.environ['AWS_ACCESS_KEY_ID'], os.environ['AWS_SECRET_ACCESS_KEY'], BUCKET_NAME)

In [ ]:
# get all files in final folder and upload
for file_path in DATA_FOLDER.iterdir():
    if file_path.stem not in ('.gitkeep'): # skip .gitkeep and other files 
        # todo- check if there is a difference between what's on the s3 and local
        mime_type, _ = mimetypes.guess_type(file_path)
        print(file_path.name, mime_type)
        s3.upload_file(f"{file_path.name}", file_path, mime_type)